In [1]:
import pandas as pd
train_file = "./data/train_data.txt"
test_file = "./data/test_data.txt"
answer_file = "./data/answer.txt"

In [2]:
%%time
df_train = pd.read_table(train_file, header=None, encoding='gb2312', sep=',')
df_test = pd.read_table(test_file, header=None, encoding='gb2312', sep=',')
df_answer = pd.read_table(answer_file, header=None, encoding='gb2312', sep=',')
df_train.rename(columns={1000:'label'}, inplace=True)
df_label = df_train['label']
df_label.shape
del df_train['label']
df_train.shape
print(df_train.shape)
print(df_label.shape)
print(df_test.shape)
print(df_answer.shape)

(80000, 1000)
(80000,)
(20000, 1000)
(20000, 1)
CPU times: user 23.8 s, sys: 3.05 s, total: 26.8 s
Wall time: 42.8 s


In [3]:
%%time
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=5000, tol=0.0000001, multi_class="ovr")
lr.fit(df_train, df_label)

predictY = lr.predict(df_test)
ac, wa = 0, 0
for py, y in zip(predictY, df_answer[0]):
    if py == y:
        ac += 1
    else:
        wa += 1
print("正确: ", ac)
print("错误: ", wa)
print("准确率: ", ac / (ac + wa))
print(lr)

正确:  16860
错误:  3140
准确率:  0.843
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=5000,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=1e-07, verbose=0,
                   warm_start=False)
CPU times: user 30 s, sys: 1.26 s, total: 31.3 s
Wall time: 22.8 s


In [ ]:
# -*- coding: UTF-8 -*-
# date: 2017-12-28
# author: emerson
from autograd import grad
import autograd.numpy as np
import sys
print(sys.version)

EPOCHS = 1000


class Adam:
    def __init__(self, loss, weights, lr=0.001, beta1=0.9, beta2=0.999, epislon=1e-8):
        self.loss = loss
        self.theta = weights
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.epislon = epislon
        self.get_gradient = grad(loss)
        self.m = 0
        self.v = 0
        self.t = 0

    def minimize_raw(self):
        self.t += 1
        g = self.get_gradient(self.theta)
        self.m = self.beta1 * self.m + (1 - self.beta1) * g
        self.v = self.beta2 * self.v + (1 - self.beta2) * (g * g)
        self.m_cat = self.m / (1 - self.beta1 ** self.t)
        self.v_cat = self.v / (1 - self.beta2 ** self.t)
        self.theta -= self.lr * self.m_cat / (self.v_cat ** 0.5 + self.epislon)

    def minimize(self):
        self.t += 1
        g = self.get_gradient(self.theta)
        lr = self.lr * (1 - self.beta2 ** self.t) ** 0.5 / \
            (1 - self.beta1 ** self.t)
        self.m = self.beta1 * self.m + (1 - self.beta1) * g
        self.v = self.beta2 * self.v + (1 - self.beta2) * (g * g)
        self.theta -= lr * self.m / (self.v ** 0.5 + self.epislon)

    def minimize_show(self, epochs=5000):
        for _ in range(epochs):
            self.t += 1
            g = self.get_gradient(self.theta)
            lr = self.lr * (1 - self.beta2 ** self.t) ** 0.5 / \
                (1 - self.beta1 ** self.t)
            self.m = self.beta1 * self.m + (1 - self.beta1) * g
            self.v = self.beta2 * self.v + (1 - self.beta2) * (g * g)
            self.theta -= lr * self.m / (self.v ** 0.5 + self.epislon)
            print("step{: 4d} g:{} lr:{} m:{} v:{} theta:{}".format(
                self.t, g, lr, self.m, self.v, self.theta))

        final_loss = self.loss(self.theta)
    # print("final loss:{} weights:{}".format(final_loss, self.theta))


def sigmoid(x):
    return 0.5*(np.tanh(x) + 1)


def logistic_predictions(weights, inputs):
    # Outputs probability of a label being true according to logistic model.
    return sigmoid(np.dot(inputs, weights))


def training_loss(weights):
    # Training loss is the negative log-likelihood of the training labels.
    preds = logistic_predictions(weights, inputs)
    label_probabilities = preds * targets + (1 - preds) * (1 - targets)
    return -np.sum(np.log(label_probabilities))


# Build a toy dataset.
inputs = np.array([[0.52, 1.12,  0.77],
                   [0.88, -1.08, 0.15],
                   [0.52, 0.06, -1.30],
                   [0.74, -2.49, 1.39]])
targets = np.array([True, True, False, True])
weights = np.array([0.0, 0.0, 0.0])


def sgd(epochs=1000):
    training_gradient_fun = grad(training_loss)
    # Optimize weights using gradient descent.
    weights = np.array([0.0, 0.0, 0.0])
    print("Initial loss:{}".format(training_loss(weights)))
    for i in range(epochs):
        weights -= training_gradient_fun(weights) * 0.01

    print("Trained loss:{}".format(training_loss(weights)))
    print("weights:{}".format(weights))


if __name__ == "__main__":
    adam = Adam(training_loss, weights, lr=0.01)
    print("start to optimize:")
    # adam.minimize_show(epochs=EPOCHS)

    import pandas as pd
    train_df = pd.read_table("./data/train_data.txt")
    train_test = pd.read_table("./data/test_data.txt")

    targets = train_df[['1000']]


    for i in range(EPOCHS):
        adam.minimize_raw()
    print("weights:{} loss:{}".format(adam.theta, adam.loss(adam.theta)))


In [13]:
import numpy as np
f  = lambda x, y: (1.5 - x + x*y)**2 + (2.25 - x + x*y**2)**2 + (2.625 - x + x*y**3)**2

def loss(weights):
    x,y = weights
    return f(x,y)


weights = np.array([1, 1.2, 1.5])
get_gradient = grad(loss)

print(weights)
get_gradient(weights)

[1.  1.2 1.5]


ValueError: too many values to unpack (expected 2)